# 1D Finite Element Solver
We aim to solve a differential equation for a 1 dimensional case with the form
$$
- \frac{\partial}{\partial x} \alpha(x) \cdot \frac{\partial \Phi (x)}{\partial x} + \beta(x) \cdot \Phi (x)=f (x)
$$
with the galerkin method. Details can be found in the [lecture notes]("G:\My Drive\HKA EITB\Semester 6\Methoden der Feldberechnung\GalerkinVerfahren.pdf"), page 13.

To do so, we split the domain of the function given as boundary values in smaller parts and solve a linear equations system comprising all elements.

## Initialise the Exercise Information

In [19]:
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import functions as myfunc

sns.set_theme()

In [20]:
def alpha(x):
    """
    Piecewise function for alpha(x):
    - 3 in [1.5, 2.7]
    - x^2 elsewhere
    """
    x = np.asarray(x)
    return np.where((1.5 <= x) & (x <= 2.7), 3.0, np.square(x))

def beta(x):
    """
    Piecewise function for beta(x):
    - x / (x+1) in [1, 2]
    - x^2 elsewhere
    """
    x = np.asarray(x)
    return np.where((1 <= x) & (x <= 2), x / (x + 1), np.square(x))

def rhs(x):
    """
    Right-hand side f(x):
    - x in [2, 4]
    - x + 1 elsewhere
    """
    x = np.asarray(x)
    return np.where((2 <= x) & (x <= 4), x, x + 1)

In [21]:
DOMAIN_BOUNDARY = {
    "Lower Bound": 1,
    "Upper Bound": 4
}
BOUNDARY_CONDITION = {
    "Lower Bound": {
        "x": DOMAIN_BOUNDARY["Lower Bound"],
        "Phi": np.exp(DOMAIN_BOUNDARY["Lower Bound"])
    },
    "Upper Bound": {
        "x": DOMAIN_BOUNDARY["Upper Bound"],
        "Phi": np.exp(DOMAIN_BOUNDARY["Upper Bound"])
    }
}

## Define Elements

In [22]:
NUMBER_OF_ELEMENTS = 5
element_nodes = np.linspace(DOMAIN_BOUNDARY["Lower Bound"], DOMAIN_BOUNDARY["Upper Bound"], num=NUMBER_OF_ELEMENTS)
print(element_nodes)

element_indices = myfunc.get_edge_indices(element_nodes)
print(element_indices)

# Append boundary dict with index position of x value
BOUNDARY_CONDITION["Lower Bound"]["x Index"] = np.where(element_nodes == BOUNDARY_CONDITION["Lower Bound"]["x"])[0][0]
BOUNDARY_CONDITION["Upper Bound"]["x Index"] = np.where(element_nodes == BOUNDARY_CONDITION["Upper Bound"]["x"])[0][0]
print(BOUNDARY_CONDITION)

[1.   1.75 2.5  3.25 4.  ]
[[0 1]
 [1 2]
 [2 3]
 [3 4]]
{'Lower Bound': {'x': 1, 'Phi': np.float64(2.718281828459045), 'x Index': np.int64(0)}, 'Upper Bound': {'x': 4, 'Phi': np.float64(54.598150033144236), 'x Index': np.int64(4)}}


## Create Domain Matrix by generating matrix values for each element

In [24]:
elements = []
for indices in element_indices:
    element = myfunc.assemble_element(element_nodes, indices[0], indices[1], alpha, beta, rhs)
    elements.append(element)

print(elements)

[{'Start Index': np.int64(0), 'End Index': np.int64(1), 'Coefficients': array([[ 2.66557018, -2.44846491],
       [-2.44846491,  2.66557018]]), 'Right Hand Side': array([0.890625, 0.890625])}, {'Start Index': np.int64(1), 'End Index': np.int64(2), 'Coefficients': array([[ 5.12890625, -3.43554688],
       [-3.43554688,  5.12890625]]), 'Right Hand Side': array([0.796875, 0.796875])}, {'Start Index': np.int64(2), 'End Index': np.int64(3), 'Coefficients': array([[13.08723958, -9.98763021],
       [-9.98763021, 13.08723958]]), 'Right Hand Side': array([1.078125, 1.078125])}, {'Start Index': np.int64(3), 'End Index': np.int64(4), 'Coefficients': array([[ 20.80598958, -15.87825521],
       [-15.87825521,  20.80598958]]), 'Right Hand Side': array([1.359375, 1.359375])}]
